In [ ]:
!pip install git+https://github.com/OML-Team/open-metric-learning.git

In [1]:
!pip uninstall -y Pillow
!pip install Pillow==9.0.0

Found existing installation: Pillow 9.0.0
Uninstalling Pillow-9.0.0:
  Successfully uninstalled Pillow-9.0.0
  Using cached Pillow-9.0.0-cp310-cp310-macosx_11_0_arm64.whl (2.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-metric-learning 0.3.1 requires pillow==9.2.0, but you have pillow 9.0.0 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip uninstall -y numpy pandas
!pip install numpy pandas

Found existing installation: numpy 1.21.6
Uninstalling numpy-1.21.6:
  Successfully uninstalled numpy-1.21.6
Found existing installation: pandas 1.1.5
Uninstalling pandas-1.1.5:
  Successfully uninstalled pandas-1.1.5
  Using cached numpy-1.23.4-cp310-cp310-macosx_11_0_arm64.whl (13.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 26.7 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-metric-learning 0.3.1 requires numpy==1.21.6, but you have numpy 1.23.4 which is incompatible.
open-metric-learning 0.3.1 requires pandas==1.1.5, but you have pandas 1.5.1 which is incompatible.
open-metric-learning 0.3.1 requires pillow==9.2.0, but you have pillow 9.0.0 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models.vit.vit import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset

dataset_root = "mock_dataset/"
df_train, _ = download_mock_dataset(dataset_root)

model = ViTExtractor("vits16_dino", arch="vits16", normalise_features=False).train()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

train_dataset = DatasetWithLabels(df_train, dataset_root=dataset_root)
criterion = TripletLossWithMiner(margin=0.1, miner=AllTripletsMiner())
sampler = BalanceSampler(train_dataset.get_labels(), n_labels=2, n_instances=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

for batch in tqdm(train_loader):
    embeddings = model(batch["input_tensors"])
    loss = criterion(embeddings, batch["labels"])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/27 [00:00<?, ?it/s]

https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth


100%|███████████████████| 2/2 [00:00<00:00,  7.25it/s]


In [2]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetQueryGallery
from oml.metrics.embeddings import EmbeddingMetrics
from oml.models.vit.vit import ViTExtractor
from oml.utils.download_mock_dataset import download_mock_dataset

dataset_root =  "mock_dataset/"
_, df_val = download_mock_dataset(dataset_root)

model = ViTExtractor("vits16_dino", arch="vits16", normalise_features=False).eval()

val_dataset = DatasetQueryGallery(df_val, dataset_root=dataset_root)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4)
calculator = EmbeddingMetrics()
calculator.setup(num_samples=len(val_dataset))

with torch.no_grad():
    for batch in tqdm(val_loader):
        batch["embeddings"] = model(batch["input_tensors"])
        calculator.update_data(batch)

metrics = calculator.compute_metrics()

Mock dataset has been downloaded already.
https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth
Checkpoint is already here.


100%|███████████████████| 3/3 [00:00<00:00, 28.85it/s]


Metrics:
{'OVERALL': {'cmc': {5: tensor(1.)},
             'map': {5: tensor(0.9375)},
             'precision': {5: tensor(1.)}}}
